In [1]:
import pandas as pd
import numpy as np

In [2]:
import spotlight

ImportError: No module named 'spotlight'

In [2]:
df_rate = pd.read_csv("data/rating_train.csv")
df = df_rate.drop("date", axis=1)
df = df.drop_duplicates()
df['label'] = 1

In [3]:
food_count = np.zeros((df.foodid.max()+1))
for i, j in zip(df.foodid.value_counts().index.values, df.foodid.value_counts().values):
    food_count[i] = j

user_count = np.zeros((df_rate.userid.max()+1))
for i, j in zip(df_rate.userid.value_counts().index.values, df_rate.userid.value_counts().values):
    user_count[i] = j

eaten = {i:set() for i in df_rate.userid}
for u, f in zip(df_rate.userid, df_rate.foodid):
    eaten[u].add(f)
    
user_food_cnt = [{} for i in range(len(user_count))]
for u, f in zip(df_rate.userid, df_rate.foodid):
    if user_food_cnt[u].get(f) != None:
        user_food_cnt[u][f] += 1
    else:
        user_food_cnt[u][f] = 1

In [6]:
u_f_mat = np.zeros((len(df_rate.userid.drop_duplicates()), len(df_rate.foodid.drop_duplicates())+1))

In [22]:
user_ids = df_rate.userid.drop_duplicates().values
for u in df_rate.userid.drop_duplicates():
    mapped_id = np.where(user_ids==u)[0][0]
    for k, v in user_food_cnt[u].items():
        u_f_mat[mapped_id][k+1] = v
    u_f_mat[mapped_id] /= np.max(u_f_mat[mapped_id])

In [25]:
import pickle
with open("caser_pytorch/user_food_cnt.pkl", "wb") as f:
    pickle.dump(u_f_mat, f)

In [61]:
all_ans = []
food_id_descent = df.foodid.value_counts().index.values
for u in df.userid.drop_duplicates():
    sorted_D = sorted(user_food_cnt[u].items(), key=lambda d: d[1], reverse=True)
    u_eaten = [x[0] for x in sorted_D[:20]]
    u_not_eaten = [i for i in food_id_descent if i not in eaten[u]]
    if u==6:
        print(u_eaten)
        print(u_not_eaten[:10])
    ans = []
    ans.extend(u_eaten[:15])
    next_idx = 0
    for i, j in enumerate(u_not_eaten):
        if len(ans) >= 20: break
        if j in u_eaten: continue
        ans.append(j)
        next_idx = i+1
    #ans.extend(u_eaten[5:])
    #for j in u_not_eaten[next_idx:]:
    #    if len(ans) >= 20: break
    #    if j in u_eaten: continue
    #    ans.append(j)
    all_ans.append(ans)

[2, 18, 10, 0, 11, 39, 3, 13, 27, 19, 7, 12, 1, 5, 6, 38, 15, 67, 9, 42]
[230, 224, 192, 178, 294, 212, 577, 322, 395, 435]


In [62]:
print(all_ans[0])

[2, 18, 10, 0, 11, 39, 3, 13, 27, 19, 7, 12, 1, 5, 6, 230, 224, 192, 178, 294]


In [49]:
all_ans[0]

[2, 18, 10, 0, 11, 39, 3, 13, 27, 19, 7, 12, 1, 5, 6, 38, 15, 67, 9, 42]

In [45]:
print(all_ans[0])

[2, 18, 10, 0, 11, 230, 224, 192, 178, 294, 39, 3, 13, 27, 19, 212, 577, 322, 395, 435]


In [63]:
with open("submission.csv", "w") as f:
    f.write("userid,foodid\n")
    for i, u in enumerate(df.userid.drop_duplicates()):
        f.write("{},".format(u))
        for j in all_ans[i]:
            f.write("{} ".format(j))
        f.write('\n')

In [65]:
import time
#!kaggle competitions submit -c ntucsie-sdml2018-2-2 -f submission.csv -m "15 eaten; 5 not"
#time.sleep(5)
!kaggle competitions submissions ntucsie-sdml2018-2-2 | more

fileName        date                 description                     status    publicScore  privateScore  
--------------  -------------------  ------------------------------  --------  -----------  ------------  
submission.csv  2018-10-25 17:13:27  15 eaten; 5 not                 complete  0.27402      None          
submission.csv  2018-10-25 17:04:18  20 eaten                        complete  0.28731      None          
submission.csv  2018-10-25 16:59:47  5 eaten; 5 not; 5 eaten; 5 not  complete  0.22852      None          
submission.csv  2018-10-25 16:33:48  most frequent                   complete  0.00371      None          
baseline.csv    2018-10-25 14:08:56  None                            complete  0.00371      None          

